In [2]:
import os,re,subprocess
import numpy as np

float_pattern = re.compile(r"[-+]?\d*\.\d+")
float_pattern_p = re.compile(r"[-+]?\d*\.\d*")

In [32]:
def get_outstreams(filename): # gets the compressed stream information at the end of a Gaussian job
    streams = []
    starts,ends = [],[]
    error = "failed or incomplete job" # default unless "normal termination" is in file
    with open(filename,"r") as f:
        filecont = f.readlines()
    for i in range(len(filecont)):
        if "1\\1\\" in filecont[i]:
            starts.append(i)
        if "@" in filecont[i]:
            ends.append(i)
        if "Normal termination" in filecont[i]:
            error = ""
    if len(starts) != len(ends) or len(starts) == 0 or error != "": 
        error = "failed or incomplete job"
        return(error)
    for i in range(len(starts)):
        stream = "".join([line.strip() for line in filecont[starts[i]:ends[i]+1]]).split("\\")
        streams.append(stream)
    return(streams)

def get_filecont(filename): # gets the entire job output
     # default unless "normal termination" is in file
    with open(filename,"r") as f:
        filecont = f.readlines()
    for line in filecont[-10:]:
        if "Normal termination" in line:
            return(filecont)
    return(["failed or incomplete job"])

def get_nmr(filecont,query): # nmr=giao
    """Return the isotropic chemical shift and chemical shift anisotropy tensor eigenvalues of atom 'query'."""
    nmrstart_pattern = "SCF GIAO Magnetic shielding tensor"
    for i in range(len(filecont)-1):
        if re.search(nmrstart_pattern,filecont[i]):
            shift_s = float(re.findall(float_pattern,filecont[i+1+query*5])[0])
            anisotropy_ev = [float(x) for x in re.findall(float_pattern,filecont[i+5+query*5])]
            return([shift_s]+anisotropy_ev)            
    return([None])

In [4]:
get_outstreams('test_giao.log')

[['1',
  '1',
  'GINC-COMPUTE-2-2',
  'SP',
  'RPBE1PBE',
  'def2TZVP',
  'C27H36N3P1',
  'STOBA2',
  '26-Jun-2024',
  '0',
  '',
  '# # PBE1PBE/def2TZVP int=(grid=ultrafine) empiricaldispersion=GD3BJ nmr=giao',
  '',
  'Complex test',
  '',
  '0,1',
  'C,0,1.605982,-2.336654,2.835589',
  'C,0,1.557563,-2.250285,1.337554',
  'C,0,2.309855,-3.159057,0.608865',
  'C,0,2.314976,-3.184138,-0.79013',
  'N,0,3.039798,-4.118057,-1.492506',
  'C,0,3.97768,-4.96865,-0.798285',
  'C,0,3.192147,-3.976115,-2.921745',
  'C,0,1.513449,-2.234618,-1.443124',
  'C,0,0.770643,-1.327108,-0.723736',
  'C,0,0.774651,-1.30717,0.66841',
  'P,0,-0.188562,-0.030148,1.586081',
  'C,0,0.602748,1.421287,0.774832',
  'C,0,0.227079,1.934368,-0.462417',
  'C,0,0.879309,3.005127,-1.034798',
  'C,0,1.956324,3.623396,-0.385419',
  'N,0,2.607057,4.696907,-0.935811',
  'C,0,2.104253,5.329165,-2.130366',
  'C,0,3.826953,5.198725,-0.352356',
  'C,0,2.33607,3.095848,0.856625',
  'C,0,1.685167,2.020486,1.431896',
  'C,0,2.15

In [5]:
get_filecont('test_giao.log')

[' Entering Gaussian System, Link 0=g16\n',
 ' Input=/home/stoba2/test_giao.gjf\n',
 ' Output=/home/stoba2/test_giao.log\n',
 ' Initial command:\n',
 ' /share/apps/gaussian/g16/l1.exe "/scratch/stoba2/225157/Gau-305298.inp" -scrdir="/scratch/stoba2/225157/"\n',
 ' Entering Link 1 = /share/apps/gaussian/g16/l1.exe PID=    305299.\n',
 '  \n',
 ' Copyright (c) 1988-2019, Gaussian, Inc.  All Rights Reserved.\n',
 '  \n',
 ' This is part of the Gaussian(R) 16 program.  It is based on\n',
 ' the Gaussian(R) 09 system (copyright 2009, Gaussian, Inc.),\n',
 ' the Gaussian(R) 03 system (copyright 2003, Gaussian, Inc.),\n',
 ' the Gaussian(R) 98 system (copyright 1998, Gaussian, Inc.),\n',
 ' the Gaussian(R) 94 system (copyright 1995, Gaussian, Inc.),\n',
 ' the Gaussian 92(TM) system (copyright 1992, Gaussian, Inc.),\n',
 ' the Gaussian 90(TM) system (copyright 1990, Gaussian, Inc.),\n',
 ' the Gaussian 88(TM) system (copyright 1988, Gaussian, Inc.),\n',
 ' the Gaussian 86(TM) system (copyrigh

In [843]:
import re

def get_nmr(filecont, atom_index):
    """Return the isotropic chemical shift, chemical shift anisotropy (CSA), and asymmetry parameter for the atom at 'atom_index'."""
    nmrstart_pattern = "SCF GIAO Magnetic shielding tensor"
    float_pattern = r"[-+]?\d*\.\d+|\d+"
    
    for i in range(len(filecont)-1):
        if re.search(nmrstart_pattern, filecont[i]):
            try:
                isotropic_shift = float(re.findall(float_pattern, filecont[i+1+atom_index*5])[0])
                eigenvalues = [float(x) for x in re.findall(float_pattern, filecont[i+5+atom_index*5])]
                
                # Sort eigenvalues
                eigenvalues.sort()
                σ11, σ22, σ33 = eigenvalues
                
                # Calculate Chemical Shift Anisotropy (CSA)
                csa = σ33 - (σ11 + σ22) / 2
                
                # Calculate asymmetry parameter
                # We use isotropic_shift instead of σiso
                asymmetry = (σ22 - σ11) / (σ33 - isotropic_shift)
                
                return {
                    "isotropic_shift": isotropic_shift,
                    "csa": csa,
                    "asymmetry": asymmetry,
                    "eigenvalues": eigenvalues
                }
            except (IndexError, ValueError) as e:
                print(f"Error: Could not find or process NMR data for atom index {atom_index}")
                print(f"Specific error: {str(e)}")
                return None
    
    print("Error: NMR data not found in the file")
    return None

# Main script
filename = "ligandsDFT_jake/l33_gjf/converted/conformer_1_optimized_out_gjffolder/conformer_1_optimized_out_giao.log"  # Replace with your actual filename
filecont = get_filecont(filename)

if filecont[0] != "failed or incomplete job":
    # Example: Get NMR data for the fifth atom (index 11)
    nmr_data = get_nmr(filecont, 11)
    
    if nmr_data is not None:
        print(f"Isotropic chemical shift: {nmr_data['isotropic_shift']:.2f} ppm")
        print(f"Chemical Shift Anisotropy (CSA): {nmr_data['csa']:.2f} ppm")
        print(f"Asymmetry parameter: {nmr_data['asymmetry']:.2f}")
        print(f"Tensor eigenvalues (xx,yy,zz): {[f'{ev:.2f}' for ev in nmr_data['eigenvalues']]}")
    else:
        print("Failed to retrieve NMR data")
else:
    print("The Gaussian job failed or is incomplete")

Isotropic chemical shift: 12.00 ppm
Chemical Shift Anisotropy (CSA): 142.32 ppm
Asymmetry parameter: 0.13
Tensor eigenvalues (xx,yy,zz): ['264.94', '321.06', '435.33']


In [307]:
import os
import re
import numpy as np
import pandas as pd

def extract_thermal_free_energy(file_path):
    """Extract the Sum of electronic and thermal Free Energies from a given .log file."""
    pattern = re.compile(r"Sum of electronic and thermal Free Energies=\s*(-?\d+\.\d+)")
    with open(file_path, 'r') as file:
        for line in file:
            match = pattern.search(line)
            if match:
                return float(match.group(1))
    return None

def get_file_content(file_path):
    """Get the content of a file as a list of lines."""
    try:
        with open(file_path, 'r') as file:
            return file.readlines()
    except FileNotFoundError:
        return ["failed or incomplete job"]

def get_efg(filecont, atom_index):
    """Return EFG tensor eigenvalues (xx, yy, zz), amplitude, and asymmetry parameter for the atom at 'atom_index'."""
    efg_pattern = "Center         ---- Electric Field Gradient ----"
    for i in range(len(filecont) - 1):
        if re.search(efg_pattern, filecont[i]) and "Eigenvalues" in filecont[i + 2]:
            try:
                efg_ev = np.array([float(val) for val in filecont[i + atom_index + 4].split()[2:5]])

                # Sort eigenvalues by absolute value, but keep their signs
                idx = np.argsort(np.abs(efg_ev))
                Vxx, Vyy, Vzz = efg_ev[idx]

                efg_ampl = np.linalg.norm(efg_ev)

                # Calculate asymmetry parameter
                asymmetry = (Vyy - Vxx) / Vzz

                return {
                    "eigenvalues": [Vxx, Vyy, Vzz],
                    "amplitude": efg_ampl,
                    "Vzz": Vzz,
                    "asymmetry": asymmetry
                }
            except (IndexError, ValueError) as e:
                print(f"Error: Could not find or process EFG data for atom index {atom_index}")
                print(f"Specific error: {str(e)}")
                return None

    print("Error: EFG data not found in the file")
    return None

def get_nmr(filecont, atom_index):
    """Return the isotropic chemical shift, chemical shift anisotropy (CSA), and asymmetry parameter for the atom at 'atom_index'."""
    nmrstart_pattern = "SCF GIAO Magnetic shielding tensor"
    float_pattern = r"[-+]?\d*\.\d+|\d+"

    for i in range(len(filecont) - 1):
        if re.search(nmrstart_pattern, filecont[i]):
            try:
                isotropic_shift = float(re.findall(float_pattern, filecont[i + 1 + atom_index * 5])[0])
                eigenvalues = [float(x) for x in re.findall(float_pattern, filecont[i + 5 + atom_index * 5])]

                # Sort eigenvalues
                eigenvalues.sort()
                σ11, σ22, σ33 = eigenvalues

                # Calculate Chemical Shift Anisotropy (CSA)
                csa = σ33 - (σ11 + σ22) / 2

                # Calculate asymmetry parameter
                # We use isotropic_shift instead of σiso
                asymmetry = (σ22 - σ11) / (σ33 - isotropic_shift)

                return {
                    "isotropic_shift": isotropic_shift,
                    "csa": csa,
                    "asymmetry": asymmetry,
                    "eigenvalues": eigenvalues
                }
            except (IndexError, ValueError) as e:
                print(f"Error: Could not find or process NMR data for atom index {atom_index}")
                print(f"Specific error: {str(e)}")
                return None

    print("Error: NMR data not found in the file")
    return None

def process_log_files(base_folder, atom_index):
    """Process all giao.log files in their respective folders and extract thermal free energies, EFG data, and NMR data."""
    results = []
    for root, dirs, files in os.walk(base_folder):
        for file in files:
            if file.endswith("_optimized_out_giao.log"):
                file_path = os.path.join(root, file)
                energy = extract_thermal_free_energy(file_path)
                conformer = re.search(r'conformer_(\d+)_optimized_out_giao.log', file).group(1)
                
                efg_path = file_path.replace("_giao.log", "_efg.log")
                filecont_efg = get_file_content(efg_path)
                
                nmr_path = file_path
                filecont_nmr = get_file_content(nmr_path)
                
                efg_data = get_efg(filecont_efg, atom_index) if filecont_efg[0] != "failed or incomplete job" else None
                nmr_data = get_nmr(filecont_nmr, atom_index) if filecont_nmr[0] != "failed or incomplete job" else None

                results.append({
                    'Conformer': conformer,
                    'File': file,
                    'Energy': energy,
                    'EFG Eigenvalues': efg_data['eigenvalues'] if efg_data else [None, None, None],
                    'EFG Amplitude': efg_data['amplitude'] if efg_data else None,
                    'Vzz': efg_data['Vzz'] if efg_data else None,
                    'EFG Asymmetry': efg_data['asymmetry'] if efg_data else None,
                    'NMR Isotropic Shift': nmr_data['isotropic_shift'] if nmr_data else None,
                    'NMR CSA': nmr_data['csa'] if nmr_data else None,
                    'NMR Asymmetry': nmr_data['asymmetry'] if nmr_data else None,
                    'NMR Eigenvalues': nmr_data['eigenvalues'] if nmr_data else [None, None, None]
                })
    return results

# Specify the base folder path containing the conformer folders and the atom index for EFG and NMR data extraction
base_folder = 'ligandsDFT_selvin/l22_gjf/converted'
atom_index = 8  # Replace with the actual atom index you are interested in

# Process the log files and extract the thermal free energies, EFG data, and NMR data
data = process_log_files(base_folder, atom_index)

# Convert the results to a DataFrame
df = pd.DataFrame(data)

# Print DataFrame columns to debug
print(df.columns)

# Sort the DataFrame by Conformer
df = df.sort_values(by='Conformer')

# Save the DataFrame to an Excel file
output_file = os.path.join(base_folder, 'thermal_free_energies_efg_nmr_data.xlsx')
df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")


Index(['Conformer', 'File', 'Energy', 'EFG Eigenvalues', 'EFG Amplitude',
       'Vzz', 'EFG Asymmetry', 'NMR Isotropic Shift', 'NMR CSA',
       'NMR Asymmetry', 'NMR Eigenvalues'],
      dtype='object')
Data has been saved to ligandsDFT_selvin/l22_gjf/converted/thermal_free_energies_efg_nmr_data.xlsx


In [1004]:
import re
import numpy as np

def get_efg(filecont, atom_index):
    """
    Return EFG tensor eigenvalues (xx, yy, zz), amplitude, and asymmetry parameter for the atom at 'atom_index'.
    """
    efg_pattern = "Center         ---- Electric Field Gradient ----"
    for i in range(len(filecont)-1):
        if re.search(efg_pattern, filecont[i]) and "Eigenvalues" in filecont[i+2]:
            try:
                efg_ev = np.array([float(val) for val in filecont[i+atom_index+4].split()[2:5]])
                
                # Sort eigenvalues by absolute value, but keep their signs
                idx = np.argsort(np.abs(efg_ev))
                Vxx, Vyy, Vzz = efg_ev[idx]
                
                efg_ampl = np.linalg.norm(efg_ev)
                
                # Calculate asymmetry parameter
                asymmetry = (Vyy - Vxx) / Vzz
                
                return {
                    "eigenvalues": [Vxx, Vyy, Vzz],
                    "amplitude": efg_ampl,
                    "Vzz": Vzz,
                    "asymmetry": asymmetry
                }
            except (IndexError, ValueError) as e:
                print(f"Error: Could not find or process EFG data for atom index {atom_index}")
                print(f"Specific error: {str(e)}")
                return None
    
    print("Error: EFG data not found in the file")
    return None

# Main script
filename = "ligandsDFT_antonio/l36_gjf/converted/conformer_2_optimized_out_gjffolder/conformer_2_optimized_out_efg.log"  # Replace with your actual filename
filecont = get_filecont(filename)

if filecont[0] != "failed or incomplete job":
    # Example: Get EFG data for the first atom (index 0)
    efg_data = get_efg(filecont, 12)
    
    if efg_data is not None:
        print(f"EFG Eigenvalues (Vxx, Vyy, Vzz): {[f'{ev:.4f}' for ev in efg_data['eigenvalues']]} a.u.")
        print(f"EFG Amplitude: {efg_data['amplitude']:.4f} a.u.")
        print(f"Vzz: {efg_data['Vzz']:.4f} a.u.")
        print(f"Asymmetry parameter: {efg_data['asymmetry']:.4f}")
    else:
        print("Failed to retrieve EFG data")
else:
    print("The Gaussian job failed or is incomplete")

EFG Eigenvalues (Vxx, Vyy, Vzz): ['-0.5984', '-0.8063', '1.4047'] a.u.
EFG Amplitude: 1.7267 a.u.
Vzz: 1.4047 a.u.
Asymmetry parameter: -0.1480


In [306]:
import os
import re
import numpy as np
import pandas as pd

def extract_thermal_free_energy(file_path):
    """Extract the Sum of electronic and thermal Free Energies from a given .log file."""
    pattern = re.compile(r"Sum of electronic and thermal Free Energies=\s*(-?\d+\.\d+)")
    with open(file_path, 'r') as file:
        for line in file:
            match = pattern.search(line)
            if match:
                return float(match.group(1))
    return None

def get_file_content(file_path):
    """Get the content of a file as a list of lines."""
    try:
        with open(file_path, 'r') as file:
            return file.readlines()
    except FileNotFoundError:
        return ["failed or incomplete job"]

def get_efg(filecont, atom_index):
    """Return EFG tensor eigenvalues (xx, yy, zz), amplitude, and asymmetry parameter for the atom at 'atom_index'."""
    efg_pattern = "Center         ---- Electric Field Gradient ----"
    for i in range(len(filecont) - 1):
        if re.search(efg_pattern, filecont[i]) and "Eigenvalues" in filecont[i + 2]:
            try:
                efg_ev = np.array([float(val) for val in filecont[i + atom_index + 4].split()[2:5]])

                # Sort eigenvalues by absolute value, but keep their signs
                idx = np.argsort(np.abs(efg_ev))
                Vxx, Vyy, Vzz = efg_ev[idx]

                efg_ampl = np.linalg.norm(efg_ev)

                # Calculate asymmetry parameter
                asymmetry = (Vyy - Vxx) / Vzz

                return {
                    "eigenvalues": [Vxx, Vyy, Vzz],
                    "amplitude": efg_ampl,
                    "Vzz": Vzz,
                    "asymmetry": asymmetry
                }
            except (IndexError, ValueError) as e:
                print(f"Error: Could not find or process EFG data for atom index {atom_index}")
                print(f"Specific error: {str(e)}")
                return None

    print("Error: EFG data not found in the file")
    return None

def process_log_files(base_folder, atom_index):
    """Process all giao.log files in their respective folders and extract thermal free energies and EFG data."""
    results = []
    for root, dirs, files in os.walk(base_folder):
        for file in files:
            if file.endswith("_optimized_out_giao.log"):
                file_path = os.path.join(root, file)
                energy = extract_thermal_free_energy(file_path)
                conformer = re.search(r'conformer_(\d+)_optimized_out_giao.log', file).group(1)
                
                efg_path = file_path.replace("_giao.log", "_efg.log")
                filecont = get_file_content(efg_path)
                
                if filecont[0] != "failed or incomplete job":
                    efg_data = get_efg(filecont, atom_index)
                else:
                    efg_data = None

                results.append({
                    'Conformer': conformer,
                    'File': file,
                    'Energy': energy,
                    'EFG Eigenvalues': efg_data['eigenvalues'] if efg_data else [None, None, None],
                    'EFG Amplitude': efg_data['amplitude'] if efg_data else None,
                    'Vzz': efg_data['Vzz'] if efg_data else None,
                    'Asymmetry': efg_data['asymmetry'] if efg_data else None
                })
    return results

# Specify the base folder path containing the conformer folders and the atom index for EFG data extraction
base_folder = 'ligandsDFT_selvin/l22_gjf/converted'
atom_index = 8  # Replace with the actual atom index you are interested in

# Process the log files and extract the thermal free energies and EFG data
data = process_log_files(base_folder, atom_index)

# Convert the results to a DataFrame
df = pd.DataFrame(data)

# Print DataFrame columns to debug
print(df.columns)

# Sort the DataFrame by Conformer
df = df.sort_values(by='Conformer')

# Save the DataFrame to an Excel file
output_file = os.path.join(base_folder, 'thermal_free_energies_and_efg_data.xlsx')
df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")


Index(['Conformer', 'File', 'Energy', 'EFG Eigenvalues', 'EFG Amplitude',
       'Vzz', 'Asymmetry'],
      dtype='object')
Data has been saved to ligandsDFT_selvin/l22_gjf/converted/thermal_free_energies_and_efg_data.xlsx


In [944]:
import os
import re
import numpy as np
import pandas as pd

def extract_thermal_free_energy(file_path):
    """Extract the Sum of electronic and thermal Free Energies from a given .log file."""
    pattern = re.compile(r"Sum of electronic and thermal Free Energies=\s*(-?\d+\.\d+)")
    with open(file_path, 'r') as file:
        for line in file:
            match = pattern.search(line)
            if match:
                return float(match.group(1))
    return None

def get_file_content(file_path):
    """Get the content of a file as a list of lines."""
    try:
        with open(file_path, 'r') as file:
            return file.readlines()
    except FileNotFoundError:
        return ["failed or incomplete job"]

def get_efg(filecont, atom_index):
    """Return EFG tensor eigenvalues (xx, yy, zz), amplitude, and asymmetry parameter for the atom at 'atom_index'."""
    efg_pattern = "Center         ---- Electric Field Gradient ----"
    for i in range(len(filecont) - 1):
        if re.search(efg_pattern, filecont[i]) and "Eigenvalues" in filecont[i + 2]:
            try:
                efg_ev = np.array([float(val) for val in filecont[i + atom_index + 4].split()[2:5]])

                # Sort eigenvalues by absolute value, but keep their signs
                idx = np.argsort(np.abs(efg_ev))
                Vxx, Vyy, Vzz = efg_ev[idx]

                efg_ampl = np.linalg.norm(efg_ev)

                # Calculate asymmetry parameter
                asymmetry = (Vyy - Vxx) / Vzz

                return {
                    "Vxx": Vxx,
                    "Vyy": Vyy,
                    "Vzz": Vzz,
                    "amplitude": efg_ampl,
                    "asymmetry": asymmetry
                }
            except (IndexError, ValueError) as e:
                print(f"Error: Could not find or process EFG data for atom index {atom_index}")
                print(f"Specific error: {str(e)}")
                return None

    print("Error: EFG data not found in the file")
    return None

def get_nmr(filecont, atom_index):
    """Return the isotropic chemical shift, chemical shift anisotropy (CSA), and asymmetry parameter for the atom at 'atom_index'."""
    nmrstart_pattern = "SCF GIAO Magnetic shielding tensor"
    float_pattern = r"[-+]?\d*\.\d+|\d+"

    for i in range(len(filecont) - 1):
        if re.search(nmrstart_pattern, filecont[i]):
            try:
                isotropic_shift = float(re.findall(float_pattern, filecont[i + 1 + atom_index * 5])[0])
                eigenvalues = [float(x) for x in re.findall(float_pattern, filecont[i + 5 + atom_index * 5])]

                # Sort eigenvalues
                eigenvalues.sort()
                σ11, σ22, σ33 = eigenvalues

                # Calculate Chemical Shift Anisotropy (CSA)
                csa = σ33 - (σ11 + σ22) / 2

                # Calculate asymmetry parameter
                # We use isotropic_shift instead of σiso
                asymmetry = (σ22 - σ11) / (σ33 - isotropic_shift)

                return {
                    "isotropic_shift": isotropic_shift,
                    "csa": csa,
                    "asymmetry": asymmetry,
                    "σ11": σ11,
                    "σ22": σ22,
                    "σ33": σ33
                }
            except (IndexError, ValueError) as e:
                print(f"Error: Could not find or process NMR data for atom index {atom_index}")
                print(f"Specific error: {str(e)}")
                return None

    print("Error: NMR data not found in the file")
    return None

def process_log_files(base_folder, atom_index):
    """Process all giao.log files in their respective folders and extract thermal free energies, EFG data, and NMR data."""
    results = []
    for root, dirs, files in os.walk(base_folder):
        for file in files:
            if file.endswith("_optimized_out_giao.log"):
                file_path = os.path.join(root, file)
                energy = extract_thermal_free_energy(file_path)
                conformer = re.search(r'conformer_(\d+)_optimized_out_giao.log', file).group(1)
                
                efg_path = file_path.replace("_giao.log", "_efg.log")
                filecont_efg = get_file_content(efg_path)
                
                nmr_path = file_path
                filecont_nmr = get_file_content(nmr_path)
                
                efg_data = get_efg(filecont_efg, atom_index) if filecont_efg[0] != "failed or incomplete job" else None
                nmr_data = get_nmr(filecont_nmr, atom_index) if filecont_nmr[0] != "failed or incomplete job" else None

                results.append({
                    'Conformer': conformer,
                    'File': file,
                    'Energy': energy,
                    'EFG Vxx': efg_data['Vxx'] if efg_data else None,
                    'EFG Vyy': efg_data['Vyy'] if efg_data else None,
                    'EFG Vzz': efg_data['Vzz'] if efg_data else None,
                    'EFG Amplitude': efg_data['amplitude'] if efg_data else None,
                    'EFG Asymmetry': efg_data['asymmetry'] if efg_data else None,
                    'NMR Isotropic Shift': nmr_data['isotropic_shift'] if nmr_data else None,
                    'NMR CSA': nmr_data['csa'] if nmr_data else None,
                    'NMR Asymmetry': nmr_data['asymmetry'] if nmr_data else None,
                    'NMR σ11': nmr_data['σ11'] if nmr_data else None,
                    'NMR σ22': nmr_data['σ22'] if nmr_data else None,
                    'NMR σ33': nmr_data['σ33'] if nmr_data else None
                })
    return results

# Specify the base folder path containing the conformer folders and the atom index for EFG and NMR data extraction
base_folder = 'ligandsDFT_antonio/l6_gjf/converted'
atom_index = 9  # Replace with the actual atom index you are interested in

# Process the log files and extract the thermal free energies, EFG data, and NMR data
data = process_log_files(base_folder, atom_index)

# Convert the results to a DataFrame
df = pd.DataFrame(data)

# Print DataFrame columns to debug
print(df.columns)

# Sort the DataFrame by Conformer
df = df.sort_values(by='Conformer')

# Save the DataFrame to an Excel file
output_file = os.path.join(base_folder, 'thermal_free_energies_efg_nmr_data.xlsx')
df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")


Index(['Conformer', 'File', 'Energy', 'EFG Vxx', 'EFG Vyy', 'EFG Vzz',
       'EFG Amplitude', 'EFG Asymmetry', 'NMR Isotropic Shift', 'NMR CSA',
       'NMR Asymmetry', 'NMR σ11', 'NMR σ22', 'NMR σ33'],
      dtype='object')
Data has been saved to ligandsDFT_antonio/l6_gjf/converted/thermal_free_energies_efg_nmr_data.xlsx


In [85]:
import numpy as np
import yaml
from morfeus import ConeAngle, SASA, Sterimol, Dispersion, Pyramidalization, BuriedVolume

def read_xyz(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    num_atoms = int(lines[0])
    elements = []
    coords = []
    
    for line in lines[2:]:
        parts = line.split()
        elements.append(parts[0])
        coords.append([float(parts[1]), float(parts[2]), float(parts[3])])
    
    return elements, np.array(coords)

def run_morfeus_xyz(xyz_file):
    elements, coords = read_xyz(xyz_file)
    
    results = {}
    
    # Find P index
    p_idx = elements.index('P')
    results['p_idx'] = p_idx
    
    # Add dummy atom
    dummy_coords = coords[p_idx] + [0, 0, 2.28]  # 2.28 Å away from P in z-direction
    coords_extended = np.vstack([coords, dummy_coords])
    elements_extended = elements + ['H']  # Use H as dummy atom
    dummy_idx = len(elements_extended) - 1
    results['selected_dummy_idx'] = -1  # As per your example

    # Sterimol
    try:
        sterimol = Sterimol(elements_extended, coords_extended, dummy_idx+1, p_idx+1)
        results['lval'] = float(sterimol.L_value)
        results['B1'] = float(sterimol.B_1_value)
        results['B5'] = float(sterimol.B_5_value)
    except:
        print("WARNING: morfeus sterimol failed")
        results['lval'] = results['B1'] = results['B5'] = None

    # SASA
    try:
        sasa = SASA(elements, coords)
        results['sasa'] = float(sasa.area)
        results['sasa_P'] = float(sasa.atom_areas[p_idx+1])
        results['sasa_volume'] = float(sasa.volume)
        results['sasa_volume_P'] = float(sasa.atom_volumes[p_idx+1])
    except:
        print("WARNING: morfeus sasa failed")
        results['sasa'] = results['sasa_P'] = results['sasa_volume'] = results['sasa_volume_P'] = None

    # Dispersion
    try:
        disp = Dispersion(elements, coords)
        results['p_int'] = float(disp.p_int)
        results['p_int_atom'] = float(disp.atom_p_int[p_idx+1])
        results['p_int_area'] = float(disp.area)
        results['p_int_atom_area'] = float(disp.atom_areas[p_idx+1])
        results['p_int_times_p_int_area'] = results['p_int'] * results['p_int_area']
        results['p_int_atom_times_p_int_atom_area'] = results['p_int_atom'] * results['p_int_atom_area']
    except:
        print("WARNING: morfeus dispersion failed")
        results['p_int'] = results['p_int_atom'] = results['p_int_area'] = results['p_int_atom_area'] = None
        results['p_int_times_p_int_area'] = results['p_int_atom_times_p_int_atom_area'] = None

    # Pyramidalization
    try:
        pyr = Pyramidalization(elements=elements, coordinates=coords, atom_index=p_idx+1)
        results['pyr_val'] = float(pyr.P)
        results['pyr_alpha'] = float(pyr.alpha)
    except:
        print("WARNING: morfeus Pyramidalization failed")
        results['pyr_val'] = results['pyr_alpha'] = None

    # BuriedVolume
    try:
        bv = BuriedVolume(elements_extended, coords_extended, dummy_idx+1, excluded_atoms=[dummy_idx+1], z_axis_atoms=[p_idx+1])
        bv.octant_analysis()
        bv.compute_distal_volume(method="buried_volume", octants=True)

        results['vbur'] = float(bv.buried_volume)
        results['vtot'] = results['vbur'] + float(bv.distal_volume)

        qvbur = np.array(list(bv.quadrants["buried_volume"].values()))
        qvtot = qvbur + np.array(list(bv.quadrants["distal_volume"].values()))

        results['qvbur_min'] = float(np.min(qvbur))
        results['qvbur_max'] = float(np.max(qvbur))
        results['qvtot_min'] = float(np.min(qvtot))
        results['qvtot_max'] = float(np.max(qvtot))

        results['max_delta_qvbur'] = float(np.max(np.abs(np.diff(qvbur, append=qvbur[0]))))
        results['max_delta_qvtot'] = float(np.max(np.abs(np.diff(qvtot, append=qvtot[0]))))

        ovbur = np.array(list(bv.octants["buried_volume"].values()))
        ovtot = ovbur + np.array(list(bv.octants["distal_volume"].values()))

        results['ovbur_min'] = float(np.min(ovbur))
        results['ovbur_max'] = float(np.max(ovbur))
        results['ovtot_min'] = float(np.min(ovtot))
        results['ovtot_max'] = float(np.max(ovtot))

        results['near_vbur'] = float(np.sum(ovbur[4:]))
        results['far_vbur'] = float(np.sum(ovbur[:4]))
        results['near_vtot'] = float(np.sum(ovtot[4:]))
        results['far_vtot'] = float(np.sum(ovtot[:4]))

    except:
        print("WARNING: morfeus BuriedVolume failed")
        for key in ['vbur', 'vtot', 'qvbur_min', 'qvbur_max', 'qvtot_min', 'qvtot_max', 
                    'max_delta_qvbur', 'max_delta_qvtot', 'ovbur_min', 'ovbur_max', 
                    'ovtot_min', 'ovtot_max', 'near_vbur', 'far_vbur', 'near_vtot', 'far_vtot']:
            results[key] = None

    return results

# Example usage
xyz_file = 'test_folder/converted/test_optimized_out.xyz'
results = run_morfeus_xyz(xyz_file)

# Save results to YAML file
with open('morfeus_results.yml', 'w') as f:
    yaml.dump(results, f, default_flow_style=False)

print("Results saved to morfeus_results.yml")

Results saved to morfeus_results.yml


In [938]:
import os
import glob
import numpy as np
import yaml
from morfeus import ConeAngle, SASA, Sterimol, Dispersion, Pyramidalization, BuriedVolume

def read_xyz(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    num_atoms = int(lines[0])
    elements = []
    coords = []
    
    for line in lines[2:]:
        parts = line.split()
        elements.append(parts[0])
        coords.append([float(parts[1]), float(parts[2]), float(parts[3])])
    
    return elements, np.array(coords)

def run_morfeus_xyz(xyz_file):
    elements, coords = read_xyz(xyz_file)
    
    results = {}
    
    # Find P index
    p_idx = elements.index('P')
    results['p_idx'] = p_idx
    
    # Add dummy atom
    dummy_coords = coords[p_idx] + [0, 0, 2.28]  # 2.28 Å away from P in z-direction
    coords_extended = np.vstack([coords, dummy_coords])
    elements_extended = elements + ['H']  # Use H as dummy atom
    dummy_idx = len(elements_extended) - 1
    results['selected_dummy_idx'] = -1  # As per your example

    # Sterimol
    try:
        sterimol = Sterimol(elements_extended, coords_extended, dummy_idx+1, p_idx+1)
        results['lval'] = float(sterimol.L_value)
        results['B1'] = float(sterimol.B_1_value)
        results['B5'] = float(sterimol.B_5_value)
    except:
        print("WARNING: morfeus sterimol failed")
        results['lval'] = results['B1'] = results['B5'] = None

    # SASA
    try:
        sasa = SASA(elements, coords)
        results['sasa'] = float(sasa.area)
        results['sasa_P'] = float(sasa.atom_areas[p_idx+1])
        results['sasa_volume'] = float(sasa.volume)
        results['sasa_volume_P'] = float(sasa.atom_volumes[p_idx+1])
    except:
        print("WARNING: morfeus sasa failed")
        results['sasa'] = results['sasa_P'] = results['sasa_volume'] = results['sasa_volume_P'] = None

    # Dispersion
    try:
        disp = Dispersion(elements, coords)
        results['p_int'] = float(disp.p_int)
        results['p_int_atom'] = float(disp.atom_p_int[p_idx+1])
        results['p_int_area'] = float(disp.area)
        results['p_int_atom_area'] = float(disp.atom_areas[p_idx+1])
        results['p_int_times_p_int_area'] = results['p_int'] * results['p_int_area']
        results['p_int_atom_times_p_int_atom_area'] = results['p_int_atom'] * results['p_int_atom_area']
    except:
        print("WARNING: morfeus dispersion failed")
        results['p_int'] = results['p_int_atom'] = results['p_int_area'] = results['p_int_atom_area'] = None
        results['p_int_times_p_int_area'] = results['p_int_atom_times_p_int_atom_area'] = None

    # Pyramidalization
    try:
        pyr = Pyramidalization(elements=elements, coordinates=coords, atom_index=p_idx+1)
        results['pyr_val'] = float(pyr.P)
        results['pyr_alpha'] = float(pyr.alpha)
    except:
        print("WARNING: morfeus Pyramidalization failed")
        results['pyr_val'] = results['pyr_alpha'] = None

    # BuriedVolume
    try:
        bv = BuriedVolume(elements_extended, coords_extended, dummy_idx+1, excluded_atoms=[dummy_idx+1], z_axis_atoms=[p_idx+1])
        bv.octant_analysis()
        bv.compute_distal_volume(method="buried_volume", octants=True)

        results['vbur'] = float(bv.buried_volume)
        results['vtot'] = results['vbur'] + float(bv.distal_volume)

        qvbur = np.array(list(bv.quadrants["buried_volume"].values()))
        qvtot = qvbur + np.array(list(bv.quadrants["distal_volume"].values()))

        results['qvbur_min'] = float(np.min(qvbur))
        results['qvbur_max'] = float(np.max(qvbur))
        results['qvtot_min'] = float(np.min(qvtot))
        results['qvtot_max'] = float(np.max(qvtot))

        results['max_delta_qvbur'] = float(np.max(np.abs(np.diff(qvbur, append=qvbur[0]))))
        results['max_delta_qvtot'] = float(np.max(np.abs(np.diff(qvtot, append=qvtot[0]))))

        ovbur = np.array(list(bv.octants["buried_volume"].values()))
        ovtot = ovbur + np.array(list(bv.octants["distal_volume"].values()))

        results['ovbur_min'] = float(np.min(ovbur))
        results['ovbur_max'] = float(np.max(ovbur))
        results['ovtot_min'] = float(np.min(ovtot))
        results['ovtot_max'] = float(np.max(ovtot))

        results['near_vbur'] = float(np.sum(ovbur[4:]))
        results['far_vbur'] = float(np.sum(ovbur[:4]))
        results['near_vtot'] = float(np.sum(ovtot[4:]))
        results['far_vtot'] = float(np.sum(ovtot[:4]))

    except:
        print("WARNING: morfeus BuriedVolume failed")
        for key in ['vbur', 'vtot', 'qvbur_min', 'qvbur_max', 'qvtot_min', 'qvtot_max', 
                    'max_delta_qvbur', 'max_delta_qvtot', 'ovbur_min', 'ovbur_max', 
                    'ovtot_min', 'ovtot_max', 'near_vbur', 'far_vbur', 'near_vtot', 'far_vtot']:
            results[key] = None

    return results

def process_folder(folder_path):
    xyz_files = glob.glob(os.path.join(folder_path, '*.xyz'))
    
    for xyz_file in xyz_files:
        results = run_morfeus_xyz(xyz_file)
        
        # Generate the output file name
        base_name = os.path.basename(xyz_file)
        output_file = os.path.join(folder_path, f'morfeus_{base_name}.yml')
        
        # Save results to YAML file
        with open(output_file, 'w') as f:
            yaml.dump(results, f, default_flow_style=False)

        print(f"Results saved to {output_file}")

# Example usage
folder_path = 'ligandsDFT/l6_gjf/converted'
process_folder(folder_path)


Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_28_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_8_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_19_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_29_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_9_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_18_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_5_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_25_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_11_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_20_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/converted/morfeus_conformer_14_optimized_out.xyz.yml
Results saved to ligandsDFT/l6_gjf/

In [607]:
import re
import numpy as np

def get_nbo_orbsP(filename):
    """Extract NBO data for phosphorus from a Gaussian output file."""
    
    float_pattern = r"[-+]?\d*\.\d+|\d+"
    
    with open(filename, 'r') as f:
        filecont = f.readlines()
    
    nbo_sum_pattern = "Natural Bond Orbitals"
    nbo_an_pattern = "NATURAL BOND ORBITAL ANALYSIS:"
    orbitals = {}
    lp_percent_s = None
    lp_occupancy = None
    
    for i, line in enumerate(filecont):
        if re.search(nbo_an_pattern, line, re.IGNORECASE):
            for j in range(i+10, len(filecont)):
                if " LP ( 1) P" in filecont[j]:
                    # Adjust regex to capture the percentage s-character correctly
                    lp_match = re.search(r"s\(\s*([\d.]+)%\)", filecont[j])
                    if lp_match:
                        lp_percent_s = float(lp_match.group(1))
                    # Adjust regex to capture the occupancy correctly
                    occ_match = re.search(r"\(\s*([\d.]+)\)", filecont[j])
                    if occ_match:
                        lp_occupancy = float(occ_match.group(1))
                    break
        
        if re.search(nbo_sum_pattern, line, re.IGNORECASE):
            for j in range(i, len(filecont)):
                if "P" in " ".join(re.findall(r"([A-Z][a-z]? *\d+)", filecont[j])).split() and ("LP" in filecont[j] or "BD" in filecont[j]):
                    orbital_desc = re.search(r"\d+\.[A-Z\*(0-9 ]+\)", filecont[j])[0] + " " + " - ".join(re.findall(r"([A-Z][a-z]? *\d+)", filecont[j]))
                    orbitals[orbital_desc] = [float(x) for x in re.findall(float_pattern, filecont[j])]  # orbital occupancy and energy
    
    if not orbitals:
        return None
    
    bd_occ = [orbitals[i][0] for i in orbitals.keys() if "BD (" in i]
    bds_occ = [orbitals[i][0] for i in orbitals.keys() if "BD*(" in i]
    bd_e = [orbitals[i][1] for i in orbitals.keys() if "BD (" in i]
    bds_e = [orbitals[i][1] for i in orbitals.keys() if "BD*(" in i]
    
    results = {
        "nbo_lp_P_percent_s": lp_percent_s,
        "nbo_lp_P_occ": lp_occupancy,
        "nbo_lp_P_e": [orbitals[i][1] for i in orbitals.keys() if "LP (" in i][0],
        "nbo_bd_e_max": max(bd_e),
        "nbo_bd_e_avg": np.mean(bd_e),
        "nbo_bds_e_min": min(bds_e),
        "nbo_bds_e_avg": np.mean(bds_e),
        "nbo_bd_occ_min": min(bd_occ),
        "nbo_bd_occ_avg": np.mean(bd_occ),
        "nbo_bds_occ_max": max(bds_occ),
        "nbo_bds_occ_avg": np.mean(bds_occ),
    }
    
    results["nbo_delta_lp_P_bds"] = results["nbo_bds_e_min"] - results["nbo_lp_P_e"]
    
    for k, v in results.items():
        results[k] = float(v)
    
    return results

# Example usage
gaussian_output_file = 'ligandsDFT_josh/l21_gjf/converted/conformer_1_optimized_out_gjffolder/conformer_1_optimized_out_nbo.log'
nbo_results = get_nbo_orbsP(gaussian_output_file)

if nbo_results:
    print("NBO Results:")
    for key, value in nbo_results.items():
        print(f"{key}: {value}")
else:
    print("Failed to extract NBO data.")

NBO Results:
nbo_lp_P_percent_s: 43.25
nbo_lp_P_occ: 1.87182
nbo_lp_P_e: 1.0
nbo_bd_e_max: 1.0
nbo_bd_e_avg: 1.0
nbo_bds_e_min: 1.0
nbo_bds_e_avg: 1.0
nbo_bd_occ_min: 25.0
nbo_bd_occ_avg: 26.0
nbo_bds_occ_max: 1237.0
nbo_bds_occ_avg: 1236.0
nbo_delta_lp_P_bds: 0.0


In [1005]:
import re
import numpy as np

def get_nbo_orbsP(filename):
    """Extract NBO data for phosphorus from a Gaussian output file."""
    
    float_pattern = r"[-+]?\d*\.\d+|\d+"
    
    with open(filename, 'r') as f:
        filecont = f.readlines()
    
    nbo_sum_pattern = "Natural Bond Orbitals"
    nbo_an_pattern = "NATURAL BOND ORBITAL ANALYSIS:"
    orbitals = {}
    lp_percent_s = None
    lp_occupancy = None
    lp_energy = None
    
    for i, line in enumerate(filecont):
        if re.search(nbo_an_pattern, line, re.IGNORECASE):
            for j in range(i+10, len(filecont)):
                if " LP ( 1) P" in filecont[j]:
                    # Adjust regex to capture the percentage s-character correctly
                    lp_match = re.search(r"s\(\s*([\d.]+)%\)", filecont[j])
                    if lp_match:
                        lp_percent_s = float(lp_match.group(1))
                    # Adjust regex to capture the occupancy correctly
                    occ_match = re.search(r"\(\s*([\d.]+)\)", filecont[j])
                    if occ_match:
                        lp_occupancy = float(occ_match.group(1))
                    # Adjust regex to capture the energy correctly
                    energy_match = re.findall(float_pattern, filecont[j])
                    if len(energy_match) >= 6:
                        lp_energy = float(energy_match[5])
                    break
        
        if re.search(nbo_sum_pattern, line, re.IGNORECASE):
            for j in range(i, len(filecont)):
                if "P" in " ".join(re.findall(r"([A-Z][a-z]? *\d+)", filecont[j])).split() and ("LP" in filecont[j] or "BD" in filecont[j]):
                    orbital_desc = re.search(r"\d+\.[A-Z\*(0-9 ]+\)", filecont[j])[0] + " " + " - ".join(re.findall(r"([A-Z][a-z]? *\d+)", filecont[j]))
                    orbitals[orbital_desc] = [float(x) for x in re.findall(float_pattern, filecont[j])]  # orbital occupancy and energy
    
    if not orbitals:
        return None

    def extract_occ_and_e(key):
        values = orbitals[key]
        return values[1], values[5] if len(values) > 5 else None

    bd_occ = []
    bds_occ = []
    bd_e = []
    bds_e = []

    for key in orbitals.keys():
        if "BD (" in key:
            occ, e = extract_occ_and_e(key)
            bd_occ.append(occ)
            if e is not None:
                bd_e.append(e)
        elif "BD*(" in key:
            occ, e = extract_occ_and_e(key)
            bds_occ.append(occ)
            if e is not None:
                bds_e.append(e)

    results = {
        "nbo_lp_P_percent_s": lp_percent_s,
        "nbo_lp_P_occ": lp_occupancy,
        "nbo_lp_P_e": lp_energy,
        "nbo_bd_e_max": max(bd_e) if bd_e else None,
        "nbo_bd_e_avg": np.mean(bd_e) if bd_e else None,
        "nbo_bds_e_min": min(bds_e) if bds_e else None,
        "nbo_bds_e_avg": np.mean(bds_e) if bds_e else None,
        "nbo_bd_occ_min": min(bd_occ) if bd_occ else None,
        "nbo_bd_occ_avg": np.mean(bd_occ) if bd_occ else None,
        "nbo_bds_occ_max": max(bds_occ) if bds_occ else None,
        "nbo_bds_occ_avg": np.mean(bds_occ) if bds_occ else None,
    }

    if results["nbo_bds_e_min"] is not None and results["nbo_lp_P_e"] is not None:
        results["nbo_delta_lp_P_bds"] = results["nbo_bds_e_min"] - results["nbo_lp_P_e"]
    else:
        results["nbo_delta_lp_P_bds"] = None

    for k, v in results.items():
        if v is not None:
            results[k] = float(v)

    return results

# Example usage
gaussian_output_file = 'ligandsDFT_antonio/l36_gjf/converted/conformer_1_optimized_out_gjffolder/conformer_1_optimized_out_nbo.log'
nbo_results = get_nbo_orbsP(gaussian_output_file)

if nbo_results:
    print("NBO Results:")
    for key, value in nbo_results.items():
        print(f"{key}: {value}")
else:
    print("Failed to extract NBO data.")


NBO Results:
nbo_lp_P_percent_s: 49.51
nbo_lp_P_occ: 1.92577
nbo_lp_P_e: 1.02
nbo_bd_e_max: -0.51652
nbo_bd_e_avg: -0.5349966666666667
nbo_bds_e_min: 0.17299
nbo_bds_e_avg: 0.19432666666666665
nbo_bd_occ_min: 1.0
nbo_bd_occ_avg: 1.0
nbo_bds_occ_max: 1.0
nbo_bds_occ_avg: 1.0
nbo_delta_lp_P_bds: -0.84701


In [1006]:
import re
import numpy as np

def get_nbo_orbsP(filename):
    """Extract NBO data for phosphorus from a Gaussian output file."""
    
    with open(filename, 'r') as f:
        filecont = f.read()
    
    print(f"File loaded. Total length: {len(filecont)} characters")
    
    # Search for the exact header
    header = "Natural Bond Orbitals (Summary):"
    index = filecont.find(header)
    
    if index != -1:
        print(f"Found '{header}' at position {index}")
        # Extract the relevant section
        summary_section = filecont[index:]
        
        # Find all lines related to phosphorus
        p_pattern = r'^\s*\d+\.\s*(BD|LP|BD\*)\s*\(\s*\d+\)\s*.*?P.*?(\d+\.\d+)\s+(-?\d+\.\d+).*?$'
        p_lines = re.findall(p_pattern, summary_section, re.MULTILINE | re.DOTALL)
        
        print(f"\nNumber of phosphorus-related lines found: {len(p_lines)}")
        
        bd_occ = []
        bds_occ = []
        bd_e = []
        bds_e = []
        lp_percent_s = None
        lp_occupancy = None
        lp_energy = None
        
        for orbital_type, occupancy, energy in p_lines:
            occupancy = float(occupancy)
            energy = float(energy)
            print(f"Parsed: Type={orbital_type}, Occupancy={occupancy}, Energy={energy}")
            
            if orbital_type == "BD":
                bd_occ.append(occupancy)
                bd_e.append(energy)
            elif orbital_type == "BD*":
                bds_occ.append(occupancy)
                bds_e.append(energy)
            elif orbital_type == "LP":
                lp_occupancy = occupancy
                lp_energy = energy
        
        # Extract s-character percentage (if available)
        s_match = re.search(r"s\(\s*([\d.]+)%\)", summary_section)
        if s_match:
            lp_percent_s = float(s_match.group(1))
        
        results = {
            "nbo_lp_P_percent_s": lp_percent_s,
            "nbo_lp_P_occ": lp_occupancy,
            "nbo_lp_P_e": lp_energy,
            "nbo_bd_e_max": max(bd_e) if bd_e else None,
            "nbo_bd_e_avg": np.median(bd_e) if bd_e else None,
            "nbo_bds_e_min": min(bds_e) if bds_e else None,
            "nbo_bds_e_avg": np.median(bds_e) if bds_e else None,
            "nbo_bd_occ_min": min(bd_occ) if bd_occ else None,
            "nbo_bd_occ_avg": np.median(bd_occ) if bd_occ else None,
            "nbo_bds_occ_max": max(bds_occ) if bds_occ else None,
            "nbo_bds_occ_avg": np.median(bds_occ) if bds_occ else None,
        }

        if results["nbo_bds_e_min"] is not None and results["nbo_lp_P_e"] is not None:
            results["nbo_delta_lp_P_bds"] = results["nbo_bds_e_min"] - results["nbo_lp_P_e"]
        else:
            results["nbo_delta_lp_P_bds"] = None

        return results
    else:
        print(f"Couldn't find '{header}' in the file")
        return None

# Example usage
gaussian_output_file = 'ligandsDFT_antonio/l36_gjf/converted/conformer_1_optimized_out_gjffolder/conformer_1_optimized_out_nbo.log'
nbo_results = get_nbo_orbsP(gaussian_output_file)
.0



0.

.
if nbo_results:
    print("\nNBO Results:")
    for key, value in nbo_results.items():
        print(f"{key}: {value}")
else:
    print("Failed to extract NBO data.")

File loaded. Total length: 672414 characters
Found 'Natural Bond Orbitals (Summary):' at position 600003

Number of phosphorus-related lines found: 11
Parsed: Type=BD, Occupancy=1.95878, Energy=-0.56738
Parsed: Type=BD, Occupancy=1.94786, Energy=-0.52109
Parsed: Type=BD, Occupancy=1.93534, Energy=-0.51652
Parsed: Type=BD, Occupancy=2.0, Energy=-76.10855
Parsed: Type=LP, Occupancy=1.92577, Energy=-0.37324
Parsed: Type=LP, Occupancy=1.90053, Energy=-0.32033
Parsed: Type=LP, Occupancy=1.89899, Energy=-0.31336
Parsed: Type=LP, Occupancy=1.89837, Energy=-0.31378
Parsed: Type=BD*, Occupancy=0.03669, Energy=0.23152
Parsed: Type=BD*, Occupancy=0.06617, Energy=0.17847
Parsed: Type=BD*, Occupancy=0.07243, Energy=0.17299

NBO Results:
nbo_lp_P_percent_s: None
nbo_lp_P_occ: 1.89837
nbo_lp_P_e: -0.31378
nbo_bd_e_max: -0.51652
nbo_bd_e_avg: -0.544235
nbo_bds_e_min: 0.17299
nbo_bds_e_avg: 0.17847
nbo_bd_occ_min: 1.93534
nbo_bd_occ_avg: 1.95332
nbo_bds_occ_max: 0.07243
nbo_bds_occ_avg: 0.06617
nbo_del

In [180]:
import re
import numpy as np

float_pattern = r"[-]?\d+\.\d+"
float_pattern_p = r"[-]?\d+\.\d+E?[-+]?\d*"

def read_log_file(filename):
    with open(filename, 'r') as file:
        return file.readlines()
def find_phosphorus_index(filecont):
    for i, line in enumerate(filecont):
        if "Input orientation:" in line or "Standard orientation:" in line:
            for j in range(i+5, len(filecont)):
                if "----" in filecont[j]:
                    break
                if " P " in filecont[j]:
                    return int(filecont[j].split()[0]) - 1  # -1 because atom indexing starts at 0
    return None

def get_nuesp(filecont, p_index):
    if p_index is None:
        return None
    nuesp_pat = "Electrostatic Properties (Atomic Units)"
    for i in range(len(filecont)-1):
        if nuesp_pat in filecont[i]:
            nuesp = float(filecont[i+6+p_index].split()[2])
            return nuesp
    return None

def get_nbo(filecont, p_index):
    if p_index is None:
        return None
    nbo_pattern = "Summary of Natural Population Analysis:"
    for i in range(len(filecont)-1):
        if re.search(nbo_pattern, filecont[i]):
            nboline = re.findall(float_pattern, filecont[i+6+p_index])
            if nboline:
                nbo = float(nboline[0])
                if len(nboline) == 6:
                    spindens = float(nboline[-1])
                    return nbo, spindens
                else:
                    return nbo
    return None
def main(filename):
    filecont = read_log_file(filename)
    
    p_index = find_phosphorus_index(filecont)
    
    results = {
        "NUESP (P atom)": get_nuesp(filecont, p_index),
        "NBO (P atom)": get_nbo(filecont, p_index),
    }
    
    for key, value in results.items():
        print(f"{key}:")
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                print(f"  {sub_key}: {sub_value}")
        else:
            print(f"  {value}")
        print()

if __name__ == "__main__":
    log_file = "ligandsDFT_selvin/l39_gjf/conformer_1.log"  # Replace with your .log file name
    main(log_file)

NUESP (P atom):
  None

NBO (P atom):
  None



In [ ]:
def get_outstreams(filename): # gets the compressed stream information at the end of a Gaussian job
    streams = []
    starts,ends = [],[]
    error = "failed or incomplete job" # default unless "normal termination" is in file
    with open(filename,"r") as f:
        filecont = f.readlines()
    for i in range(len(filecont)):
        if "1\\1\\" in filecont[i]:
            starts.append(i)
        if "@" in filecont[i]:
            ends.append(i)
        if "Normal termination" in filecont[i]:
            error = ""
    if len(starts) != len(ends) or len(starts) == 0 or error != "": 
        error = "failed or incomplete job"
        return(error)
    for i in range(len(starts)):
        stream = "".join([line.strip() for line in filecont[starts[i]:ends[i]+1]]).split("\\")
        streams.append(stream)
    return(streams)

def get_quadrupole(streams,blank):    
    """Return a 4-member list with the xx,yy,zz eigenvalues and the amplitude of the quadrupole moment tensor."""
    for item in streams[-1]:
        if "Quadrupole" in item:
            q = [float(i) for i in re.findall(float_pattern_p,item)]
            q_comps = np.array(([q[0],q[3],q[4]],[q[3],q[1],q[5]],[q[4],q[5],q[2]]))
            q_diag = np.linalg.eig(q_comps)[0]
            q_ampl = np.linalg.norm(q_diag)
            q_results = [q_ampl,np.max(q_diag),-(np.max(q_diag)+np.min(q_diag)),np.min(q_diag)]
            return(q_results)
    return([None])

def get_dipole(streams,blank):
    """Return the absolute dipole moment in Debye."""
    for item in streams[-1]:
        if "Dipole" in item:
            d_vec = [float(i) for i in re.findall(float_pattern_p,item)]
            d_abs = np.linalg.norm(d_vec) * 2.541746  # conversion from Bohr-electron to Debye
            return([d_abs])
    return([None])       


filename = "ligandsDFT_jake/l32_gjf/conformer_1.log"
streams = get_outstreams(filename)
blank=None
quadrupole_results = get_quadrupole(streams, blank)
dipole_results = get_dipole(streams,blank)

print ("qpole Amplitude, XX, YY, ZZ")
print (quadrupole_results)
print ("Dipole")
print (dipole_results)


def get_filecont(filename): # gets the entire job output
     # default unless "normal termination" is in file
    with open(filename,"r") as f:
        filecont = f.readlines()
    for line in filecont[-10:]:
        if "Normal termination" in line:
            return(filecont)
    return(["failed or incomplete job"])

def get_homolumo(filecont,blank): # homo,lumo energies and derived values of last job in file  
    homo_pattern = "Alpha  occ. eigenvalues"
    osmo_pattern = "Beta  occ. eigenvalues"
    lumo = 100
    for i in range(len(filecont)-1,0,-1):
        if re.search(osmo_pattern,filecont[i]) and lumo == 100:
            lumo = float(re.findall(float_pattern,filecont[i+1])[0])
        if re.search(homo_pattern,filecont[i]):
            homo = float(re.findall(float_pattern,filecont[i])[-1]) # in open shell systems, this is the SOMO
            lumo = min((lumo,float(re.findall(float_pattern,filecont[i+1])[0])))
            mu =  (homo+lumo)/2 # chemical potential / negative of molecular electronegativity
            eta = lumo-homo     # hardness/softness
            omega = mu**2/(2*eta) # electrophilicity index
            return([homo,lumo,mu,eta,omega])
    return([None])

def get_nbo(filecont,query): 
    """Return the NBO partial charge of atom 'query'."""
    nbo_pattern = "Summary of Natural Population Analysis:"
    for i in range(len(filecont)-1):
        if re.search(nbo_pattern,filecont[i]):
            nboline = re.findall(float_pattern,filecont[i+6+query])
            nbo = float(nboline[0])
            if len(nboline) == 6:
                spindens = float(nboline[-1])
                return([nbo,spindens])
            else:
                return([nbo])
    return([None])

filename = "ligandsDFT_jake/l33_gjf/conformer_4.log"
filecont = get_filecont(filename)
blank=None
homolumo_results = get_homolumo(filecont, blank)
nbo_results = get_nbo(filecont,7)

print ("homo, lumo, mu, eta, omega")
print (homolumo_results)

qpole Amplitude, XX, YY, ZZ
[8.66635315316932, 5.675430316121349, 0.8221969467693908, -6.49762726289074]
Dipole
[2.6775738637980107]
homo, lumo, mu, eta, omega
[-0.18182, -0.04715, -0.114485, 0.13467, 0.048662713391995245]


In [941]:
import os
import re
import numpy as np
import pandas as pd

def get_outstreams(filename):
    """Gets the compressed stream information at the end of a Gaussian job."""
    streams = []
    starts, ends = [], []
    error = "failed or incomplete job"  # default unless "normal termination" is in file
    with open(filename, "r") as f:
        filecont = f.readlines()
    for i in range(len(filecont)):
        if "1\\1\\" in filecont[i]:
            starts.append(i)
        if "@" in filecont[i]:
            ends.append(i)
        if "Normal termination" in filecont[i]:
            error = ""
    if len(starts) != len(ends) or len(starts) == 0 or error != "":
        error = "failed or incomplete job"
        return error
    for i in range(len(starts)):
        stream = "".join([line.strip() for line in filecont[starts[i]:ends[i] + 1]]).split("\\")
        streams.append(stream)
    return streams

def get_quadrupole(streams, blank):
    """Return a 4-member list with the xx,yy,zz eigenvalues and the amplitude of the quadrupole moment tensor."""
    float_pattern_p = r"[-+]?\d*\.\d+|\d+"
    for item in streams[-1]:
        if "Quadrupole" in item:
            q = [float(i) for i in re.findall(float_pattern_p, item)]
            q_comps = np.array(([q[0], q[3], q[4]], [q[3], q[1], q[5]], [q[4], q[5], q[2]]))
            q_diag = np.linalg.eig(q_comps)[0]
            q_ampl = np.linalg.norm(q_diag)
            q_results = [q_ampl, np.max(q_diag), -(np.max(q_diag) + np.min(q_diag)), np.min(q_diag)]
            return q_results
    return [None, None, None, None]

def get_dipole(streams, blank):
    """Return the absolute dipole moment in Debye."""
    float_pattern_p = r"[-+]?\d*\.\d+|\d+"
    for item in streams[-1]:
        if "Dipole" in item:
            d_vec = [float(i) for i in re.findall(float_pattern_p, item)]
            d_abs = np.linalg.norm(d_vec) * 2.541746  # conversion from Bohr-electron to Debye
            return [d_abs]
    return [None]

def get_filecont(filename):
    """Gets the entire job output."""
    with open(filename, "r") as f:
        filecont = f.readlines()
    for line in filecont[-10:]:
        if "Normal termination" in line:
            return filecont
    return ["failed or incomplete job"]

def get_homolumo(filecont, blank):
    """Returns HOMO, LUMO energies and derived values of last job in file."""
    homo_pattern = "Alpha  occ. eigenvalues"
    osmo_pattern = "Beta  occ. eigenvalues"
    float_pattern = r"[-+]?\d*\.\d+|\d+"
    lumo = 100
    for i in range(len(filecont) - 1, 0, -1):
        if re.search(osmo_pattern, filecont[i]) and lumo == 100:
            lumo = float(re.findall(float_pattern, filecont[i + 1])[0])
        if re.search(homo_pattern, filecont[i]):
            homo = float(re.findall(float_pattern, filecont[i])[-1])  # in open shell systems, this is the SOMO
            lumo = min((lumo, float(re.findall(float_pattern, filecont[i + 1])[0])))
            mu = (homo + lumo) / 2  # chemical potential / negative of molecular electronegativity
            eta = lumo - homo  # hardness/softness
            omega = mu**2 / (2 * eta)  # electrophilicity index
            return [homo, lumo, mu, eta, omega]
    return [None, None, None, None, None]

def process_log_files(base_folder):
    """Process all conformer log files and extract required data."""
    results = []
    for root, dirs, files in os.walk(base_folder):
        for file in files:
            if file.endswith(".log") and "conformer_" in file:
                file_path = os.path.join(root, file)
                print(f"Processing file: {file}")  # Debugging print statement
                match = re.search(r'conformer_(\d+)\.log', file)
                if match:
                    conformer = match.group(1)
                    
                    streams = get_outstreams(file_path)
                    filecont = get_filecont(file_path)
                    
                    if streams != "failed or incomplete job":
                        quadrupole_results = get_quadrupole(streams, None)
                        dipole_results = get_dipole(streams, None)
                    else:
                        quadrupole_results = [None, None, None, None]
                        dipole_results = [None]

                    if filecont[0] != "failed or incomplete job":
                        homolumo_results = get_homolumo(filecont, None)
                    else:
                        homolumo_results = [None, None, None, None, None]

                    results.append({
                        'Conformer': conformer,
                        'File': file,
                        'Qpole Amplitude': quadrupole_results[0],
                        'Qpole XX': quadrupole_results[1],
                        'Qpole YY': quadrupole_results[2],
                        'Qpole ZZ': quadrupole_results[3],
                        'Dipole': dipole_results[0],
                        'HOMO': homolumo_results[0],
                        'LUMO': homolumo_results[1],
                        'Mu': homolumo_results[2],
                        'Eta': homolumo_results[3],
                        'Omega': homolumo_results[4]
                    })
    return results

# Specify the base folder path containing the conformer log files
base_folder = 'ligandsDFT_antonio/l6_gjf'

# Process the log files and extract the required data
data = process_log_files(base_folder)

# Convert the results to a DataFrame
df = pd.DataFrame(data)

# Sort the DataFrame by Conformer
df = df.sort_values(by='Conformer')

# Save the DataFrame to an Excel file
output_file = os.path.join(base_folder, 'conformer_data.xlsx')
df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")


Processing file: conformer_14.log
Processing file: conformer_28.log
Processing file: conformer_29.log
Processing file: conformer_15.log
Processing file: conformer_17.log
Processing file: conformer_16.log
Processing file: conformer_12.log
Processing file: conformer_13.log
Processing file: conformer_11.log
Processing file: conformer_10.log
Processing file: conformer_9.log
Processing file: conformer_8.log
Processing file: conformer_3.log
Processing file: conformer_2.log
Processing file: conformer_1.log
Processing file: conformer_5.log
Processing file: conformer_4.log
Processing file: conformer_6.log
Processing file: conformer_7.log
Processing file: conformer_21.log
Processing file: conformer_20.log
Processing file: conformer_22.log
Processing file: conformer_23.log
Processing file: conformer_27.log
Processing file: conformer_26.log
Processing file: conformer_18.log
Processing file: conformer_24.log
Processing file: conformer_25.log
Processing file: conformer_19.log
Processing file: confor

In [312]:
import os
import yaml
import pandas as pd

def read_yaml_files(folder_path):
    data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.yml'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                file_data = yaml.safe_load(file)
                file_data['file_name'] = file_name  # Add file name to track origin
                data.append(file_data)
    return data

def write_to_excel(data, output_file):
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)
    return df

# Main logic
folder_path = 'ligandsDFT/l22_gjf/converted'  # Change this to your folder path
output_file = os.path.join(folder_path, 'output.xlsx')

data = read_yaml_files(folder_path)
df = write_to_excel(data, output_file)

# Display the DataFrame
#df


In [942]:
import os
import re
import yaml
import pandas as pd

def read_yaml_files(folder_path):
    data = []
    files = os.listdir(folder_path)
    # Sort files based on the numeric value in the file name
    sorted_files = sorted(files, key=lambda x: int(re.search(r'conformer_(\d+)', x).group(1)) if re.search(r'conformer_(\d+)', x) else float('inf'))

    for file_name in sorted_files:
        if file_name.endswith('.yml'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                file_data = yaml.safe_load(file)
                file_data['file_name'] = file_name  # Add file name to track origin
                data.append(file_data)
    return data

def write_to_excel(data, output_file):
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)
    return df

# Main logic
folder_path = 'ligandsDFT/l6_gjf/converted'  # Change this to your folder path
output_file = os.path.join(folder_path, 'output.xlsx')

data = read_yaml_files(folder_path)
df = write_to_excel(data, output_file)

# Display the DataFrame
# print(df)


In [387]:
import os
import re

def extract_thermal_free_energies(folder_path):
    # Regular expression to match the line containing the thermal free energy
    pattern = re.compile(r"Sum of electronic and thermal Free Energies=\s*(-?\d+\.\d+)")
    
    # Dictionary to store the results
    energies = {}

    # Loop through all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".log"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                # Read the file line by line
                for line in file:
                    match = pattern.search(line)
                    if match:
                        # Extract the energy value and store it with the filename
                        energies[filename] = float(match.group(1))
                        break
    
    return energies

# Specify the folder path containing the .log files
folder_path = 'ligandsDFT_jake/l27_gjf'

# Extract the thermal free energies
thermal_free_energies = extract_thermal_free_energies(folder_path)

# Sort the energies dictionary by the file names
sorted_energies = dict(sorted(thermal_free_energies.items()))

# Print the sorted results
for filename, energy in sorted_energies.items():
    print(f"File: {filename}, Sum of electronic and thermal Free Energies: {energy}")


File: conformer_1.log, Sum of electronic and thermal Free Energies: -1383.535809
File: conformer_10.log, Sum of electronic and thermal Free Energies: -1383.536197
File: conformer_11.log, Sum of electronic and thermal Free Energies: -1383.528749
File: conformer_12.log, Sum of electronic and thermal Free Energies: -1383.542179
File: conformer_13.log, Sum of electronic and thermal Free Energies: -1383.52776
File: conformer_14.log, Sum of electronic and thermal Free Energies: -1383.534493
File: conformer_15.log, Sum of electronic and thermal Free Energies: -1383.534494
File: conformer_16.log, Sum of electronic and thermal Free Energies: -1383.535805
File: conformer_17.log, Sum of electronic and thermal Free Energies: -1383.53532
File: conformer_18.log, Sum of electronic and thermal Free Energies: -1383.529152
File: conformer_19.log, Sum of electronic and thermal Free Energies: -1383.527761
File: conformer_2.log, Sum of electronic and thermal Free Energies: -1383.541964
File: conformer_20.l

In [943]:
import os
import re
import openpyxl
from openpyxl.styles import Font

def extract_thermal_free_energies(folder_path):
    pattern = re.compile(r"Sum of electronic and thermal Free Energies=\s*(-?\d+\.\d+)")
    energies = {}

    for filename in os.listdir(folder_path):
        if filename.endswith(".log"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                for line in file:
                    match = pattern.search(line)
                    if match:
                        energies[filename] = float(match.group(1))
                        break
    
    return energies

def write_to_excel(energies, output_file):
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Thermal Free Energies"

    # Write headers
    ws['A1'] = "Filename"
    ws['B1'] = "Sum of electronic and thermal Free Energies"
    ws['A1'].font = Font(bold=True)
    ws['B1'].font = Font(bold=True)

    # Write data
    for row, (filename, energy) in enumerate(energies.items(), start=2):
        ws.cell(row=row, column=1, value=filename)
        ws.cell(row=row, column=2, value=energy)

    # Adjust column widths
    ws.column_dimensions['A'].width = 30
    ws.column_dimensions['B'].width = 40

    # Save the workbook
    wb.save(output_file)

# Specify the folder path containing the .log files
folder_path = 'ligandsDFT_antonio/l6_gjf'

# Extract the thermal free energies
thermal_free_energies = extract_thermal_free_energies(folder_path)

# Sort the energies dictionary by the file names
sorted_energies = dict(sorted(thermal_free_energies.items()))

# Specify the output Excel file in the same folder as the .log files
output_file = os.path.join(folder_path, 'thermal_free_energies.xlsx')

# Write the results to Excel
write_to_excel(sorted_energies, output_file)

print(f"Results have been written to {output_file}")

Results have been written to ligandsDFT_antonio/l6_gjf/thermal_free_energies.xlsx
